In [ ]:
%%writefile mock_ledger.py
import hashlib
from dataclasses import dataclass
from typing import Dict, List


@dataclass
class Tx:
    round_id: int
    client_id: str
    payload: bytes      # here: flattened or encrypted update
    tx_hash: str


class MockBlockchain:
    """
    Very simple in-memory 'blockchain' for your prototype.

    - submit_update(round_id, client_id, payload)
    - get_round_updates(round_id)
    - store_aggregate(round_id, payload)
    - get_aggregate(round_id)
    """

    def __init__(self):
        # round_id -> {client_id -> Tx}
        self.updates: Dict[int, Dict[str, Tx]] = {}
        # round_id -> aggregate Tx
        self.aggregates: Dict[int, Tx] = {}

    # ---------- client-side API ----------

    def submit_update(self, round_id: int, client_id: str, payload: bytes):
        """Store a client update as a 'transaction' in the ledger."""
        tx_hash = hashlib.sha256(payload).hexdigest()
        tx = Tx(round_id=round_id, client_id=client_id,
                payload=payload, tx_hash=tx_hash)

        if round_id not in self.updates:
            self.updates[round_id] = {}

        self.updates[round_id][client_id] = tx
        print(f"[Ledger] Stored update: round={round_id}, client={client_id}, hash={tx_hash[:8]}...")

    def get_round_updates(self, round_id: int) -> List[bytes]:
        """Aggregator reads all client updates for a given round."""
        if round_id not in self.updates:
            return []
        return [tx.payload for tx in self.updates[round_id].values()]

    # ---------- aggregator-side API ----------

    def store_aggregate(self, round_id: int, payload: bytes):
        """Store aggregated update or global model reference."""
        tx_hash = hashlib.sha256(payload).hexdigest()
        tx = Tx(round_id=round_id, client_id="aggregator",
                payload=payload, tx_hash=tx_hash)
        self.aggregates[round_id] = tx
        print(f"[Ledger] Stored aggregate: round={round_id}, hash={tx_hash[:8]}...")

    def get_aggregate(self, round_id: int) -> bytes:
        """Retrieve stored aggregate payload for a given round."""
        return self.aggregates[round_id].payload


Writing mock_ledger.py


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Save the file permanently in Drive
!mv mock_ledger.py /content/drive/MyDrive/


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
